In [ ]:
import numpy as np
# from submit import my_map
# from submit import my_fit
import time as tm

In [ ]:
# SUBMIT.PY FILE #

import numpy as np
import sklearn
from scipy.linalg import khatri_rao
from sklearn.linear_model import LogisticRegression


def my_fit( X_train, y_train ):
	features = my_map(X_train)
	labels = y_train
	model = LogisticRegression(C = 100, max_iter = 500, penalty = 'l2', solver = 'lbfgs', tol = 0.01)
	model.fit(features, labels)
	w = model.coef_.flatten()
	b = model.intercept_[0]
	return w, b

def my_map( X ):
	X = np.flip(np.cumprod(np.flip(2 * X - 1, axis = 1), axis = 1), axis = 1)
	X = np.hstack((X, np.ones((X.shape[0], 1))))
	# X = np.concatenate((X, np.ones((X.shape[0], 1))), axis=1)
	num_columns = X.shape[1]
	feat = np.empty((X.shape[0], num_columns * (num_columns - 1) // 2))
	idx = 0
	for i in range(num_columns):
		feat[:, idx:idx+num_columns-i-1] = X[:, i+1:] * X[:, i][:, np.newaxis]
		idx += num_columns - i - 1
	return feat

In [ ]:
Z_trn = np.loadtxt( "secret_train.dat" )
Z_tst = np.loadtxt( "secret_test.dat" )

n_trials = 5

d_size = 0
t_train = 0
t_map = 0
acc = 0

In [ ]:
for t in range( n_trials ):
	tic = tm.perf_counter()
	w, b = my_fit( Z_trn[:, :-1], Z_trn[:,-1] )
	toc = tm.perf_counter()
	t_train += toc - tic

	d_size += w.shape[0]

	tic = tm.perf_counter()
	feat = my_map( Z_tst[:, :-1] )
	toc = tm.perf_counter()
	t_map += toc - tic

	scores = feat.dot( w ) + b
	pred = np.zeros_like( scores )
	pred[scores > 0] = 1
	acc += np.average( Z_tst[ :, -1 ] == pred )

In [ ]:
d_size /= n_trials
t_train /= n_trials
t_map /= n_trials
acc /= n_trials

print( d_size, t_train, t_map, 1- acc )

528.0 1.720048247199884 0.07185956200046348 0.006900000000000128
